# Lab 3 - Decision Trees

This assignment uses a dataset obtained from the JSE Data Archive that aims at identifying whether there is a statistically significant dependence between gender and biological/activity traits in humans. 

FEATURE DESCRIPTIONS: 

<ul>
<li>Color (Blue, Brown, Green, Hazel, Other)
<li>Age (in years)
<li>YearinSchool (First, Second, Third, Fourth, Other)
<li>Height (in inches)
<li>Miles (distance from home town of student to Ames, IA) 
<li>Brothers (number of brothers)
<li>Sisters (number of sisters)
<li>CompTime (number of hours spent on computer per week)
<li>Exercise (whether the student exercises Yes or No)
<li>ExerTime (number of hours spent exercising per week)
<li>MusicCDs (number of music CDs student owns)
<li>PlayGames (number of hours spent playing games per week)
<li>WatchTV (number of hours spent watching TV per week
</ul>

https://ww2.amstat.org/publications/jse/jse_data_archive.htm

In [1]:
from collections import Counter, defaultdict
from itertools import combinations 
import pandas as pd
import numpy as np
import operator

In [104]:
df = pd.read_csv('Eye_Color.csv')

In [105]:
# remove NA's and reset the index
df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df = df.reset_index(drop=True)

In [106]:
df.head()

gender  age    year eyecolor  height  miles  brothers  sisters  \
0  female   18   first    hazel    68.0  195.0         0        1   
1    male   20   third    brown    70.0  120.0         3        0   
2  female   18   first    green    67.0  200.0         0        1   
3    male   23  fourth    hazel    74.0  140.0         1        1   
4  female   19  second     blue    62.0   60.0         0        1   

   computertime exercise  exercisehours  musiccds  playgames  watchtv  
0          20.0      Yes            3.0      75.0        6.0     18.0  
1          24.0       No            0.0      50.0        0.0      3.0  
2          35.0      Yes            3.0      53.0        8.0      1.0  
3           5.0      Yes           25.0      50.0        0.0      7.0  
4           5.0      Yes            4.0      30.0        2.0      5.0

# Calculating Gini Index 

$Gini(D) = 1 - \sum_{i=1}^m p_i^2$

In [95]:
def gini(l):
    count = np.sum(l)
    gini = 1
    for p in l:
        gini -= np.square(p/count)
    return gini 

$Gini_A(D) = \frac{|D_1|}{|D|}Gini(D_1) + \frac{|D_2|}{|D|}Gini(D_2)$

In [96]:
def gini_split(l_1, l_2):
    total = np.sum(l_1) + np.sum(l_2)
    return (np.sum(l_1)/total)*gini(l_1) + (np.sum(l_2)/total)*gini(l_2)

$\Delta Gini(A) = Gini(D) - Gini_A(D)$

In [51]:
def gini_delta():
    print('TODO')



**Question 1: How many rows are there in the dataset for males? For females? **



In [108]:
df.groupby('gender').size()

gender
female    1078
male       910
dtype: int64

**Question 2: What is the Gini Index of this dataset, using males and females as the target classes?**

In [100]:
print(gini(df.groupby('gender').size().tolist()))

0.4964292799047808


## Best Split of a Numeric Feature

**Question 3: What is the best split point of the 'height' feature. **

In [119]:
min_height = int(df['height'].min())
max_height = int(df['height'].max())

partitioning = []
for height in range(min_height, max_height):
    tall = df_filtered = df.query('height>'+str(height))
    small = df_filtered = df.query('height<='+str(height))
    
    d_1 = tall.groupby('gender').size().tolist()
    d_2 = small.groupby('gender').size().tolist()
    
    partitioning.append({
        'height': height,
        'partitions': [
            d_1,
            d_2
        ],
        'gini': gini_split(d_1, d_2)
    })

best_partitioning = partitioning[0]
for p in partitioning:
    if p['gini'] < best_partitioning['gini']:
        best_partitioning = p
        
print(best_partitioning) 

{'height': 68, 'partitions': [[173, 768], [905, 142]], 'gini': 0.26552881207029194}


**Question 4: What is the Gini Index of this best split?**

In [112]:
best_partitioning['gini']

0.26552881207029194

**Question 5: How much does this partitioning reduce the Gini Index over that of the overall dataset?**

In [114]:
delta = gini(df.groupby('gender').size().tolist()) - best_partitioning['gini']
print(delta)

0.23090046783448887


**Question 6: How many 'female' rows are below your best split point? 'male' rows?**

In [123]:
print('Below 68.0 :: Female:', best_partitioning['partitions'][1][0], '/ Male:', best_partitioning['partitions'][1][1])

Below 68.0 :: Female: 905 / Male: 142
Over 68.0 :: Female: 173 / Male: 768


**Question 7: How many 'female' rows are above your best split point? 'male' rows?**

In [124]:
print('Above 68.0 :: Female:', best_partitioning['partitions'][0][0], '/ Male:', best_partitioning['partitions'][0][1])

Above 68.0 :: Female: 173 / Male: 768


Recall that, to calculate the best split of this numeric field, you'll need to order your data by 'height', then consider the midpoint between each pair of consecutive heights as a potential split point, then calculate the Gini Index for that partitioning. You'll want to keep track of the best split point and its Gini Index (remember that you are trying to minimize the Gini Index). 

There are a lot of ways to do this. Some are very fast, others very slow. One tip to make this run quickly is, as you consecutively step through the data and calculate the Gini Index of each possible split point, keep a running total of the number of rows for each candidate that are located above and below the split point. 

Some Python tips: 

* Counter(), from the collections module, is a special dictionary for counting values of a key
* zip() lets you concatenate lists into a list of tuples (for example, if we have a list of the candidates and a list of transaction amounts, zip(candidate_list, transaction_amount) would give us a list of (candidate, transaction amount) pairs

## Best Split of a Categorial Variable

**Question 8: How many possible splits are there of the eyecolor feature?**

In [148]:
import itertools

# Question: 5 values, how can they be partitioned?
#   - The order of elements in each partition (aka branch of the tree) does not matter.
#   - The order of partitions does not matter.
#   - There can be up to 5 branches, even though we want to predict only 2 targets. 

# 5 branches (1 option) => (5 choose 5 = 1)
    #(1,1,1,1,1)
        # [(1), (2), (3), (4), (5)]
    
# 4 branches (10 options) 
    # (2,1,1,1) => (5 choose 3 = 10)
        # [(1,2), (3), (4), (5)]
        # [(1,3), (2), (4), (5)]
        # [(1,4), (2), (3), (5)]
        # [(1,5), (2), (3), (4)]
    
        # [(1), (2, 3), (4), (5)]
        # [(1), (2, 4), (3), (5)]
        # [(1), (2, 5), (3), (4)]
    
        # [(1), (2), (3, 4), (5)]
        # [(1), (2), (3, 5), (4)]
    
        # [(1), (2), (3), (4,5)]
    
# 3 branches (10 options) 
    # (3, 1, 1)  => (5 choose 2 = 10)
        # [(1,2,3), (4), (5)]
        # [(1,2,4), (3), (5)]
        # [(1,2,5), (3), (4)]
    
        # [(1,3,4), (2), (5)]
        # [(1,3,5), (2), (4)]

        # [(1,4,5), (2), (3)]
    
        # [(2,3,4), (1), (5)]
        # [(2,3,5), (1), (4)]
    
        # [(2,4,5), (1), (3)]
    
        # [(3,4,5), (1), (2)]
        
    #(2,2,1)  => (?? = 15 options)
        # [(1), (2,3), (4,5)]
        # [(1), (2,4), (3,5)]
        # [(1), (2,5), (3,4)]
        
        # [(2), (1,3), (4,5)]
        # [(2), (1,4), (3,5)]
        # [(2), (1,5), (3,4)]
        
        # [(3), (1,2), (4,5)]
        # [(3), (1,4), (2,5)]
        # [(3), (1,5), (2,4)]
        
        # [(4), (1,2), (3,5)]
        # [(4), (1,3), (2,5)]
        # [(4), (1,5), (2,3)]
        
        # [(5), (1,2), (3,4)]
        # [(5), (1,3), (2,4)]
        # [(5), (1,4), (2,3)]
        
# 2 branches (5 options)
    # (4,1)  => (5 choose 1 = 5)
        # [(1), (2,3,4,5)]
        # [(2), (1,3,4,5)] 
        # [(3), (1,2,4,5)]
        # [(4), (1,2,3,5)]
        # [(5), (1,2,3,4)]
    # (2,3)  => (5 choose 2 = 10)
        # [(1,2),(3,4,5)]
        # [(1,3),(2,4,5)]
        # [(1,4),(2,3,5)]
        # [(1,5),(2,3,4)]
        # [(2,3),(1,4,5)]
        # [(2,4),(1,3,5)]
        # [(2,5),(1,3,4)]
        # [(3,4),(1,2,5)]
        # [(3,5),(1,2,4)]
        # [(4,5),(1,2,3)]
        
# 1 branche (1 option) // ignore
    # (5)  => (5 choose 0 = 1)
        # [(1,2,3,4,5)] (useless, though)
    


values = ['Blue', 'Brown', 'Green', 'Hazel', 'Other']
#       [   1   ,    2   ,   3    ,   4   ,     5   ]

combinations = [
    #(1,1,1,1,1)
    [(1), (2), (3), (4), (5)],
    
    #(2,1,1,1)
    [(1,2), (3), (4), (5)],
    [(1,3), (2), (4), (5)],
    [(1,4), (2), (3), (5)],
    [(1,5), (2), (3), (4)],
    [(1), (2,3), (4), (5)],
    [(1), (2,4), (3), (5)],
    [(1), (2,5), (3), (4)],
    [(1), (2), (3,4), (5)],
    [(1), (2), (3,5), (4)],
    [(1), (2), (3), (4,5)],
    
    #(3,1,1)
    [(1,2,3), (4), (5)],
    [(1,2,4), (3), (5)],
    [(1,2,5), (3), (4)],
    [(1,3,4), (2), (5)],
    [(1,3,5), (2), (4)],
    [(1,4,5), (2), (3)],
    [(2,3,4), (1), (5)],
    [(2,3,5), (1), (4)],
    [(2,4,5), (1), (3)],
    [(3,4,5), (1), (2)],
    
    #(1,2,2)
    [(1), (2,3), (4,5)],
    [(1), (2,4), (3,5)],
    [(1), (2,5), (3,4)],
    [(2), (1,3), (4,5)],
    [(2), (1,4), (3,5)],
    [(2), (1,5), (3,4)],
    [(3), (1,2), (4,5)],
    [(3), (1,4), (2,5)],
    [(3), (1,5), (2,4)],
    [(4), (1,2), (3,5)],
    [(4), (1,3), (2,5)],
    [(4), (1,5), (2,3)],
    [(5), (1,2), (3,4)],
    [(5), (1,3), (2,4)],
    [(5), (1,4), (2,3)],
    
    #(1,4)
    [(1), (2,3,4,5)],
    [(2), (1,3,4,5)] ,
    [(3), (1,2,4,5)],
    [(4), (1,2,3,5)],
    [(5), (1,2,3,4)],
    
    #(2,3)
    [(1,2),(3,4,5)],
    [(1,3),(2,4,5)],
    [(1,4),(2,3,5)],
    [(1,5),(2,3,4)],
    [(2,3),(1,4,5)],
    [(2,4),(1,3,5)],
    [(2,5),(1,3,4)],
    [(3,4),(1,2,5)],
    [(3,5),(1,2,4)],
    [(4,5),(1,2,3)],
    
    #(5) //ignore because this is the same as not using the feature at all
    #[(1,2,3,4,5)]

]

# print(len(combinations)) # // 51


from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

for result in powerset([1, 2, 3, 4,5]):
    print(result)

results = list(powerset([1, 2, 3, 4, 5]))

# print(len(results)) // 32
#   (+1)  - does not ignore (5), which is useless 
#   (+5)  - The (1,1,1,1,1) set is counted 6 times
#   (-xx) - ??
#   (-15) - Does not account for a (1,2,2) split
#   => 32 - 1 - 5 + 15 == 41


# Note: Could you please share the code of how to generate the list of combinations with itertools?
# I could not find a way to systematically generate all partitions, but would be very 
# interested in the solutions to this problem.

    

()
(1,)
(2,)
(3,)
(4,)
(5,)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(2, 3)
(2, 4)
(2, 5)
(3, 4)
(3, 5)
(4, 5)
(1, 2, 3)
(1, 2, 4)
(1, 2, 5)
(1, 3, 4)
(1, 3, 5)
(1, 4, 5)
(2, 3, 4)
(2, 3, 5)
(2, 4, 5)
(3, 4, 5)
(1, 2, 3, 4)
(1, 2, 3, 5)
(1, 2, 4, 5)
(1, 3, 4, 5)
(2, 3, 4, 5)
(1, 2, 3, 4, 5)


**Question 9: Which split of eyecolor best splits the female and male rows, as measured by the Gini Index?**

**Question 10: What is the Gini Index of this best split?**

**Question 11: How much does this partitioning reduce the Gini Index over that of the overall data set?**

**Question 12: How many 'female' rows and 'male' rows are in your first partition? How many 'female' rows and 'male' rows are in your second partition?**

Python tip: the combinations function of the itertools module allows you to enumerate combinations of a list. You might want to Google 'power set'.

## Training a decision tree

**Question 13: Using all of the features in the original dataframe read in at the top of this notebook, train a decision tree classifier that has a depth of three (including the root node and leaf nodes). What is the accuracy of this classifier on the training data?**

Scikit-learn classifiers require class labels and features to be in numeric arrays. As such, you will need to turn your categorical features into numeric arrays using DictVectorizer. This is a helpful notebook for understanding how to do this: http://nbviewer.ipython.org/gist/sarguido/7423289. You can turn a pandas dataframe of features into a dictionary of the form needed by DictVectorizer by using df.to_dict('records'). Make sure you remove the class label first (in this case, gender). If you use the class label as a feature, your classifier will have a training accuracy of 100%! The example notebook link also shows how to turn your class labels into a numeric array using sklearn.preprocessing.LabelEncoder().

In [ ]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer #to turn categorial variables into numeric arrays
from sklearn import preprocessing #to transform the feature labels

**Question 14: Using the following code snippet, export your decision tree to graphviz and visualize it. In your write-up, write down the interpretation of the rule at each node which is used to perform the splitting.**

In order to install graphviz, you may need to download the tool from [this website](https://graphviz.gitlab.io), and then pip3/conda install the python libraries you do not have.

Mac users can use ```brew install graphviz``` instead of following the link, and linux users can do the same using their favourite package manager (for example, Ubuntu users can use ```sudo apt-get install graphviz```, followed by the necessary pip3/conda installations.

In [10]:
from IPython.display import Image  
import pydotplus 
import pydot
from sklearn.externals.six import StringIO

#
# clf = your classifier

dotfile = StringIO() 
tree.export_graphviz(clf, out_file=dotfile,
#                     feature_names=df.columns,  
#                          class_names=['Female', 'Male'],  
                         filled=True, rounded=True,  
                         special_characters=True)
                    
graph = pydotplus.graph_from_dot_data(dotfile.getvalue())
Image(graph.create_png())

**Question 15 (Extra Credit): For each of your leaf nodes, specify the percentage of 'female' rows in that node (out of the total number of rows at that node).**

See this notebook for the basics of training a decision tree in scikit-learn and exporting the outputs to view in graphviz: http://nbviewer.ipython.org/gist/tebarkley/b68c04d9b31e64ce6023